# Purpose
The goal of this notebook is to make a master CSV. 



In [1461]:
import pandas as pd
import re
import numpy as np
from dateutil import parser

## Functions

In [1462]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \((.*)\)"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
def change_percent(test):
    if isinstance(test, str):
        query = r"%"
        m = re.search(query, test)

        if m:
            test = test.strip("%")
            test = float(test)
            return test
        else:
            return test
    else:
        return test
    
def get_state(test):
    query = r"([A-Z][A-Z])\Z"
    m = re.search(query, test)
    return m.group()


### Bringing in Pending 2014 Wait Times

In [1463]:
# A function to format 
def make_df(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,24,25,26],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Pending_1412 = make_df("Pending","1412","Data/Wait_Time/Pending_Clean/14_12_01_Wait_Pending.csv")
Pending_1412.head()

,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,Station
0,"(V01) (402) Togus, ME",42899,2.70%,2.51,4.07,2.77,402
1,"(V01) (405) White River Junction, VT",23709,2.16%,3.92,3.95,0.79,405
2,"(V01) (518) Bedford, MA",8105,5.17%,1.12,10.53,4.41,518
3,"(V01) (523) VA Boston HCS, MA",85625,1.71%,1.66,3.53,3.92,523
4,"(V01) (608) Manchester, NH",19511,2.64%,2.66,4.21,5.13,608


# Staff Size

In [1464]:
staff = pd.read_csv("Data/Staff_Size/Onboard_By_statyion_By_FY.csv")
staff = staff.drop(0).drop(1) # dropping first and second lines because these don't refer to actual stations.
staff["Station"] = staff["ORGANIZATION"].apply(get_station)
staff.head()
#staff

,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,Station
2,(V01) (402) HCS TOGUS ME,1293,1272.0,1248,1283,1335,1446,402
3,(V01) (405) MROC WHT RIVER JCT VT,852,853.0,881,921,1022,1177,405
4,(V01) (478) V1HCSBEDFORD MA,30,36.0,34,53,48,42,478
5,(V01) (518) MC BEDFORD MA,1313,1265.0,1245,1251,1297,1352,518
6,(V01) (523) HCS BOSTON MA,4022,4011.0,4041,4032,4123,4303,523


### Trying To Consolidate Staff And Pending

In [1465]:
#Grabbing all stations in pending that have duplicates
multiple_pending_series = Pending_1412["Station"].value_counts()
multiple_pending_dataframe = multiple_pending_series.to_frame()
multiple_pending_list = multiple_pending_dataframe[multiple_pending_dataframe["Station"] > 1].index.tolist()

pending_unique = pd.DataFrame()
for item in multiple_pending_list:
    new = Pending_1412[Pending_1412["Station"]==item]
    pending_unique = pending_unique.append(new)
    
    
#grabbing all in staff that aren't in pending...  
in_staff_not_pending = [x for x in staff["Station"].tolist() if x not in Pending_1412["Station"].tolist()]
staff_unique = pd.DataFrame()
for item in in_staff_not_pending:
    new = staff[staff["Station"]==item]
    staff_unique = staff_unique.append(new)
    
# ...and including that in a dataframe with the uniques from 
for item in multiple_pending_list:
    new = staff[staff["Station"]==item]
    staff_unique = staff_unique.append(new)



#### Cleaning
Identified areas that need to be clean and stations that match. At this point I'm going to try and generate unique IDs and then I'm going to bring in other data to join. 

Many of the locations in station in the hiring csv don't match the wait time csv. 


In [1466]:
# identified these as not stations, but facilities. Dropping from dataframe. 
to_drop = [18,29,50,59,66,77,87,96,105,114,123,144,152]
for index in to_drop:
    Pending_1412 = Pending_1412.drop(index)



### Generating Unique IDs

In [1467]:
IDs = []
query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"
#m = re.search(query, test)
#m.group(3)



for index, row in Pending_1412.iterrows():
    if row["Station"] not in IDs:
        m = re.search(query, row["Location"])
        loc = m.group(3).lower()
        IDs.append(row["Station"])

    else:
        m = re.search(query, row["Location"])
        loc = m.group(3).lower()
        IDs.append(row["Station"]+loc)
Pending_1412["ID"] = IDs
Pending_1412.head()

,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,Station,ID
0,"(V01) (402) Togus, ME",42899,2.70%,2.51,4.07,2.77,402,402
1,"(V01) (405) White River Junction, VT",23709,2.16%,3.92,3.95,0.79,405,405
2,"(V01) (518) Bedford, MA",8105,5.17%,1.12,10.53,4.41,518,518
3,"(V01) (523) VA Boston HCS, MA",85625,1.71%,1.66,3.53,3.92,523,523
4,"(V01) (608) Manchester, NH",19511,2.64%,2.66,4.21,5.13,608,608


In [1468]:
#manually changing some IDs in staff based on shared location
staff["ID"] = staff["Station"]
staff["ID"][11] = "528"
staff["ID"][92] = "657st"
staff["ID"][12] = "528bu"
staff.head()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,Station,ID
2,(V01) (402) HCS TOGUS ME,1293,1272.0,1248,1283,1335,1446,402,402
3,(V01) (405) MROC WHT RIVER JCT VT,852,853.0,881,921,1022,1177,405,405
4,(V01) (478) V1HCSBEDFORD MA,30,36.0,34,53,48,42,478,478
5,(V01) (518) MC BEDFORD MA,1313,1265.0,1245,1251,1297,1352,518,518
6,(V01) (523) HCS BOSTON MA,4022,4011.0,4041,4032,4123,4303,523,523


In [1469]:
master = Pending_1412.merge(staff, on="ID", how="left")
master["Station"] = master["Station_x"]
master= master.drop("Station_x",1).drop("Station_y",1)
print master.columns

Index([u'Location', u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Station'],
      dtype='object')


#### So what was the result of our merge?

In [1470]:
def check_len(master):
    check = len(master) - 141
    if check == 0:
        print "Hurray! We didn't lose any rows."
    if check < 0:
        print "Uh oh. We lost {} rows!".format(str(abs(check)))
    if check > 0:
        print "we somehow...gained {} row? Well that's not right.".format(str(abs(check)))
check_len(master)

Hurray! We didn't lose any rows.


Great! Now I'm going to redefine ID so that it better matches with future dataframes.

In [1471]:
ID = []
query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"

for index, row in master.iterrows():
    m = re.search(query, row["Location"])
    loc = m.group(3).lower()
    ID.append(row["Station"]+loc)

master["ID"] = ID
master["ID"][72] = "537je" # edge case
master.head()



/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,Station
0,"(V01) (402) Togus, ME",42899,2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,1283.0,1335.0,1446.0,402
1,"(V01) (405) White River Junction, VT",23709,2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,921.0,1022.0,1177.0,405
2,"(V01) (518) Bedford, MA",8105,5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,1251.0,1297.0,1352.0,518
3,"(V01) (523) VA Boston HCS, MA",85625,1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,4032.0,4123.0,4303.0,523
4,"(V01) (608) Manchester, NH",19511,2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,686.0,759.0,803.0,608


## Bringing in Pending 2016

In [1472]:
#functions to format 2016 
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,20,21,22],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Pending_1610 = make_df_2016("Pending","1610","Data/Wait_Time/Pending_Clean/16_10_01_Wait_Pending.csv")
#Pending_1610 = Pending_1610.drop(297).drop(369) # edge case causing issues, not a station anyway

In [1473]:
def generate_IDs(df):
    ID = []
    for index, row in df.iterrows():
        #print index
        if len(row["Station"]) > 3:
            #print row["Station"]
            ID.append(None)
        else:
            try:
                #print index
                #print "entering else"
                #print "This is the location: "+row["Location"]
                query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"
                m = re.search(query, row["Location"].upper())
                new_ID = row["Station"]+m.group(3).lower()
                if new_ID not in ID:
                    ID.append(new_ID)
                else:
                    ID.append(None)
                #print m.group(3).upper()
            except AttributeError:
                #print "AttributeError"
                ID.append(None)
    return ID
ID = generate_IDs(Pending_1610)
Pending_1610["ID"] = ID

In [1474]:
Pending_1610 = Pending_1610.dropna(subset = ["ID"])
master = master.merge(Pending_1610,on="ID")
print master.columns
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)
master.head()

Index([u'Location_x', u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Station_x', u'Location_y', u'Appts_Pending_1610',
       u'%_Appts_Over_30_Pending_1610', u'PC_Wait_Pending_1610',
       u'SC_Wait_Pending_1610', u'MH_Wait_Pending_1610', u'Station_y'],
      dtype='object')


,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,Appts_Pending_1610,%_Appts_Over_30_Pending_1610,PC_Wait_Pending_1610,SC_Wait_Pending_1610,MH_Wait_Pending_1610,Station,Location
0,42899,2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,1283.0,1335.0,1446.0,43609,4.18%,7.82,5.67,3.62,402,"(V01) (402) Togus, ME"
1,23709,2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,921.0,1022.0,1177.0,21091,3.83%,6.09,7.09,1.86,405,"(V01) (405) White River Junction, VT"
2,8105,5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,1251.0,1297.0,1352.0,14456,7.54%,0.60,11.21,3.07,518,"(V01) (518) Bedford, MA"
3,85625,1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,4032.0,4123.0,4303.0,84373,4.43%,14.46,6.76,3.08,523,"(V01) (523) VA Boston HCS, MA"
4,19511,2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,686.0,759.0,803.0,25424,4.09%,5.75,4.58,5.21,608,"(V01) (608) Manchester, NH"


In [1475]:
check_len(master)

Hurray! We didn't lose any rows.


## Bringing in Complete 2016

In [1476]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,13,14,15],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Complete_1608 = make_df_2016("Complete","1608","Data/Wait_Time/Completed_Cleaned/16_08_31_Wait_Complete.csv")
Complete_1608.head()

,Location,Appts_Complete_1608,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Wait_Complete_1608,Station
0,"(V01) (402) Togus, ME",28683,0.0134,4.96,2.73,1.87,402
1,(V01) (402) Togus VAMC,17464,0.0163,5.96,2.66,2.59,402
2,(V01) (402GA) Aroostook County\r(Caribou),756,0.0423,8.84,0.00,3.41,402GA
3,(V01) (402GB) Calais,379,0.0000,1.43,0.00,0.36,402GB
4,(V01) (402GC) Rumford,428,0.0000,1.31,0.00,1.11,402GC


In [1477]:
len(Complete_1608)

1189

In [1478]:
ID = generate_IDs(Complete_1608)
Complete_1608["ID"] = ID
Complete_1608 = Complete_1608.dropna(subset = ["ID"])
master = master.merge(Complete_1608,on="ID")
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)

In [1479]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location'],
      dtype='object')

In [1480]:
check_len(master)

Hurray! We didn't lose any rows.


In [1481]:
# Reference for duplicates I need to check out. 
master[master.duplicated(subset="Station", keep=False)][["Location","Station","ID"]]

,Location,Station,ID
8,"(V02) (528) Albany, NY",528,528al
9,"(V02) (528) Bath, NY",528,528ba
10,"(V02) (528) Western New York, NY",528,528we
11,"(V02) (528) Canandaigua, NY",528,528ca
12,"(V02) (528) Syracuse, NY",528,528sy
79,"(V15) (589) Columbia, MO",589,589co
80,"(V15) (589) Kansas City, MO",589,589ka
81,"(V15) (589) Eastern KS HCS, KS",589,589ea
82,"(V15) (589) Wichita, KS",589,589wi
83,"(V15) (657) Marion, IL",657,657ma


# Bring in November Complete 2016

In [1482]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,13,14,15],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv

# Complete_1611 = make_df_2016("Complete","1611","Wait_Time/Completed_Cleaned/16_11_Wait.csv")

# ID = generate_IDs(Complete_1611)
# Complete_1611["ID"] = ID
# Complete_1611 = Complete_1611.dropna(subset = ["ID"])
# master = master.merge(Complete_1611,on="ID",how='left')
# master["Station"] = master["Station_x"]
# master["Location"] = master["Location_x"]
# master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)


# Bring In November Pending 2016

## Bring in Hiring

I'm going to have an issue dealing with the stations with the same ID. 

I made a new CSV called Hiring_Clean where I put the appropriate ID in a new ID column. I put ignore for those that were impossible to join accurately. 

In [1483]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location'],
      dtype='object')

In [1484]:
#Hiring = pd.read_csv("Hiring/Hiring_Clean.csv")
#Hiring = Hiring[Hiring["ID"]!= "Ignore"] 
#Hiring_summed = pd.pivot_table(Hiring,index=["Station"],values=["NbrEmps"],aggfunc=np.sum).reset_index()
#Hiring_summed = Hiring_summed.rename(columns = {"NbrEmps":"Choice_Hires"})

In [1485]:
#Hiring = pd.read_csv("Hiring/Hiring_Clean.csv")
Hiring = pd.read_csv("Data/Hiring/Hiring_Clean_All_Positions.csv")
for_dups = []
for index, row in Hiring.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Hiring["for_dups"] = for_dups
Hiring_summed = pd.pivot_table(Hiring,index=["for_dups"],values=["NbrEmps"],aggfunc=np.sum).reset_index()
Hiring_summed = Hiring_summed.rename(columns = {"NbrEmps":"Choice_Hires"})


dup_list = master[master.duplicated(subset="Station",keep=False)]["ID"].tolist()
dup_list.append("612n.")
dup_list.append("537je")

for_dups = []
for index, row in master.iterrows():
    if row["ID"] in dup_list:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
master["for_dups"] = for_dups

In [1486]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location', u'for_dups'],
      dtype='object')

In [1487]:
master = master.merge(Hiring_summed, on="for_dups", how="left")
master.head()

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,MH_Wait_Pending_1610,Appts_Complete_1608,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Wait_Complete_1608,Station,Location,for_dups,Choice_Hires
0,42899,2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,3.62,28683,0.0134,4.96,2.73,1.87,402,"(V01) (402) Togus, ME",402,47.0
1,23709,2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,1.86,21581,0.0177,3.62,4.08,1.32,405,"(V01) (405) White River Junction, VT",405,47.0
2,8105,5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,3.07,13170,0.0115,0.43,6.01,0.76,518,"(V01) (518) Bedford, MA",518,13.0
3,85625,1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,3.08,46306,0.0271,7.47,5.75,2.17,523,"(V01) (523) VA Boston HCS, MA",523,86.0
4,19511,2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,5.21,18849,0.0160,4.99,2.93,2.94,608,"(V01) (608) Manchester, NH",608,48.0


In [1488]:
check_len(master)

Hurray! We didn't lose any rows.


## Bring in Leadership

Also creating a version of the CSV to deal with duplicate stations.

In [1489]:
Leadership = pd.read_csv("Leadership/Leadership_cleaned.csv")
Leadership = Leadership[Leadership["ID"] != "Ignore"]
Leadership["Vacancy"] = Leadership["Acting/Detailed"] == "VACANT"
Leadership["Interim"] = Leadership["Acting/Detailed"] != "VACANT"


# For easy merging


for_dups = []
for index, row in Leadership.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Leadership["for_dups"] = for_dups

Leadership_summed = pd.pivot_table(Leadership, index=["for_dups"],values=["Vacancy","Interim"],aggfunc=np.sum).reset_index()
#Leadership_summed = Leadership_summed.merge(Leadership[["Station","ID"]], on="Station")












In [1490]:
master = master.merge(Leadership_summed, on="for_dups", how="left")
master["Interim"] = master["Interim"].fillna(0)
master["Vacancy"] = master["Vacancy"].fillna(0)

In [1491]:
check_len(master)

Hurray! We didn't lose any rows.


In [1492]:
master.head()

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Wait_Complete_1608,Station,Location,for_dups,Choice_Hires,Interim,Vacancy
0,42899,2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,0.0134,4.96,2.73,1.87,402,"(V01) (402) Togus, ME",402,47.0,0.0,1.0
1,23709,2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,0.0177,3.62,4.08,1.32,405,"(V01) (405) White River Junction, VT",405,47.0,0.0,0.0
2,8105,5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,0.0115,0.43,6.01,0.76,518,"(V01) (518) Bedford, MA",518,13.0,0.0,1.0
3,85625,1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,0.0271,7.47,5.75,2.17,523,"(V01) (523) VA Boston HCS, MA",523,86.0,0.0,0.0
4,19511,2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,0.0160,4.99,2.93,2.94,608,"(V01) (608) Manchester, NH",608,48.0,0.0,0.0


In [1493]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location', u'for_dups', u'Choice_Hires', u'Interim', u'Vacancy'],
      dtype='object')

## Bringing in Completed 2014

In [1494]:
pd.read_csv("Data/Wait_Time/Completed_Cleaned/14_09_30_Wait_Complete.csv").head()

,Location,Total Appts Completed1,Appts completed in 30 Days or under2,Percent Appts completed in 30 Days or under3,Appts completed in over 30 Days4,Percent Appts completed in over 30 Days5,Appts Completed Between 0 - 14 Days6,Appts Completed Between 15 - 30 Days7,Appts Completed Between 31 - 60 days8,Appts Completed Between 61 - 90 Days9,Appts Completed Between 91 - 120 Days10,Appts Completed Beyond 120 Days11,PC Avg Wait Time in Days12,SC Avg Wait Time in Days13,MH Avg Wait Time in Days14
0,"(V01) (402) Togus, ME",24458,24050,0.9833,408,0.0167,"23,433",617,327,57,16,8,2.72,3.34,1.43
1,"(V01) (405) White River Junction, VT",19232,18929,0.9842,303,0.0158,"18,419",510,250,40,9,4,2.24,3.22,0.84
2,"(V01) (518) Bedford, MA",11976,11786,0.9841,190,0.0159,"11,357",429,166,18,6,0,0.88,5.30,3.67
3,"(V01) (523) VA Boston HCS, MA",45669,45188,0.9895,481,0.0105,"43,900","1,288",379,77,16,9,1.16,3.17,2.63
4,"(V01) (608) Manchester, NH",17616,17329,0.9837,287,0.0163,"16,714",615,246,39,2,0,2.49,3.65,3.01


In [1495]:
def make_df_complete_2014(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,12,13,14],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Complete_1409 = make_df_complete_2014("Complete","1409","Data/Wait_Time/Completed_Cleaned/14_09_30_Wait_Complete.csv")
Complete_1409["ID"] = generate_IDs(Complete_1409)
Complete_1409

,Location,Appts_Complete_1409,%_Appts_Over_30_Complete_1409,PC_Wait_Complete_1409,SC_Wait_Complete_1409,MH_Wait_Complete_1409,Station,ID
0,"(V01) (402) Togus, ME",24458,0.0167,2.72,3.34,1.43,402,402to
1,"(V01) (405) White River Junction, VT",19232,0.0158,2.24,3.22,0.84,405,405wh
2,"(V01) (518) Bedford, MA",11976,0.0159,0.88,5.30,3.67,518,518be
3,"(V01) (523) VA Boston HCS, MA",45669,0.0105,1.16,3.17,2.63,523,523va
4,"(V01) (608) Manchester, NH",17616,0.0163,2.49,3.65,3.01,608,608ma
5,(V01) (631) VA Central Western Massachusetts HCS,19536,0.0500,6.31,7.80,2.91,631,631va
6,"(V01) (650) Providence, RI",28521,0.0142,2.35,3.65,4.05,650,650pr
7,"(V01) (689) VA Connecticut HCS, CT",48622,0.0150,1.34,3.42,2.99,689,689va
8,"(V02) (528) Albany, NY",24865,0.0088,1.28,1.86,3.02,528,528al
9,"(V02) (528) Bath, NY",13553,0.0185,3.05,5.33,0.13,528,528ba


In [1496]:
for_dups = []
for index, row in Complete_1409.iterrows():
    if row["ID"] in dup_list:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Complete_1409["for_dups"] = for_dups

In [1497]:
columns_to_use = Complete_1409.columns.difference(master.columns).tolist()
columns_to_use.append("for_dups")

master = master.merge(Complete_1409[columns_to_use], on="for_dups", how="left")
master.columns






Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location', u'for_dups', u'Choice_Hires', u'Interim', u'Vacancy',
       u'%_Appts_Over_30_Complete_1409', u'Appts_Complete_1409',
       u'MH_Wait_Complete_1409', u'PC_Wait_Complete_1409',
       u'SC_Wait_Complete_1409'],
      dtype='object')

In [1498]:
master.head()

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,Location,for_dups,Choice_Hires,Interim,Vacancy,%_Appts_Over_30_Complete_1409,Appts_Complete_1409,MH_Wait_Complete_1409,PC_Wait_Complete_1409,SC_Wait_Complete_1409
0,42899,2.70%,2.51,4.07,2.77,402to,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,"(V01) (402) Togus, ME",402,47.0,0.0,1.0,0.0167,24458,1.43,2.72,3.34
1,23709,2.16%,3.92,3.95,0.79,405wh,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,"(V01) (405) White River Junction, VT",405,47.0,0.0,0.0,0.0158,19232,0.84,2.24,3.22
2,8105,5.17%,1.12,10.53,4.41,518be,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,"(V01) (518) Bedford, MA",518,13.0,0.0,1.0,0.0159,11976,3.67,0.88,5.30
3,85625,1.71%,1.66,3.53,3.92,523va,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,"(V01) (523) VA Boston HCS, MA",523,86.0,0.0,0.0,0.0105,45669,2.63,1.16,3.17
4,19511,2.64%,2.66,4.21,5.13,608ma,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,"(V01) (608) Manchester, NH",608,48.0,0.0,0.0,0.0163,17616,3.01,2.49,3.65


In [1499]:
check_len(master)    

Hurray! We didn't lose any rows.


# Turnover

In [1500]:
turnover = pd.read_csv("Data/Turnover/Turnover_Physician.csv",na_values="-")
# to drop national numbers and visn
turnover = turnover[turnover["Organization"].str.len() > 5]

turnover["Station"]= turnover["Organization"].apply(get_station)

for_dups = []
for index, row in turnover.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
turnover["for_dups"] = for_dups


turnover.head()

,Organization,Specialty,FY11,FY12,FY13,FY14,FY15,ID,Station,for_dups
174,(V01) (402) HCS TOGUS ME,0602 Physician (All Specialties),0.0997,0.0955,0.1133,0.1474,0.0781,NaN,402,402
175,(V01) (402) HCS TOGUS ME,01 ANESTHESIOLOGY,0.3750,0.7742,NaN,NaN,NaN,NaN,402,402
176,(V01) (402) HCS TOGUS ME,02 SURGERY,NaN,NaN,NaN,0.6667,NaN,NaN,402,402
177,(V01) (402) HCS TOGUS ME,07 ORTHOPEDIC SURGERY,NaN,0.2927,NaN,NaN,NaN,NaN,402,402
178,(V01) (402) HCS TOGUS ME,08 OTOLARYNGOLOGY,NaN,NaN,NaN,1.0000,NaN,NaN,402,402


### Not merging in all of turnover

To do this would create a massive number of columns. Instead, it makes more sense to merge filtered versions in on a case by case basis.

But...it does make sense to merge in specialty 

In [1501]:
turnover.to_csv("Data/turnover/Turnover_Physician_For_Merging.csv")

In [1502]:

physician_turnover = pd.read_csv("Data/Turnover/Turnover_Physician_For_Merging.csv",na_values="-",
                           usecols=[2,3,4,5,6,7,10],
                           skiprows=[0], #skipping the original header
                           names=["Specialty","Physician_Turnover_FY11","Physician_Turnover_FY12",
                                  "Physician_Turnover_FY13","Physician_Turnover_FY14","Physician_Turnover_FY15",
                                  "for_dups"])
physician_turnover = physician_turnover[physician_turnover["Specialty"]=="0602 Physician (All Specialties)"].drop("Specialty",1)
master = master.merge(physician_turnover,on="for_dups",how="left")

In [1503]:
master[master["Station"]=="672"]

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,%_Appts_Over_30_Complete_1409,Appts_Complete_1409,MH_Wait_Complete_1409,PC_Wait_Complete_1409,SC_Wait_Complete_1409,Physician_Turnover_FY11,Physician_Turnover_FY12,Physician_Turnover_FY13,Physician_Turnover_FY14,Physician_Turnover_FY15
51,125488,1.16%,2.42,2.61,3.33,672sa,(V08) (672) MC SAN JUAN PR,3346.0,3309.0,3337.0,...,0.0056,77005,2.03,1.12,1.08,0.0568,0.0427,0.0195,0.0396,0.016


# Looking at pyschiatrists

In [1504]:
psychiatrists_hiring = pd.read_csv("Data/Hiring/Hiring_Psychiatrist.csv")
psychiatrists_hiring
for_dups = []
for index, row in psychiatrists_hiring.iterrows():
    if type(row["ID"]) == str:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
psychiatrists_hiring["for_dups"] = for_dups
psychiatrists_hiring["NbrPsychiatrists"] = psychiatrists_hiring["NbrEmps"]

master = master.merge(psychiatrists_hiring[["NbrPsychiatrists","for_dups"]], on="for_dups", how="left")


# Fixing Percents

In [1505]:
master["%_Appts_Over_30_Complete_1409"] = master["%_Appts_Over_30_Complete_1409"]*100
master["%_Appts_Over_30_Complete_1608"] = master["%_Appts_Over_30_Complete_1608"]*100
master[master["Station"]=="672"]

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,ID,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,Appts_Complete_1409,MH_Wait_Complete_1409,PC_Wait_Complete_1409,SC_Wait_Complete_1409,Physician_Turnover_FY11,Physician_Turnover_FY12,Physician_Turnover_FY13,Physician_Turnover_FY14,Physician_Turnover_FY15,NbrPsychiatrists
51,125488,1.16%,2.42,2.61,3.33,672sa,(V08) (672) MC SAN JUAN PR,3346.0,3309.0,3337.0,...,77005,2.03,1.12,1.08,0.0568,0.0427,0.0195,0.0396,0.016,2.0


In [1506]:
master = master.applymap(change_percent)

# Self Generated Columns

In [1507]:
# Appts
master["Pending_Increase"] = master["Appts_Pending_1610"] - master["Appts_Pending_1412"]
master["Pending_Increase_%"] = (master["Pending_Increase"]/master["Appts_Pending_1412"])*100
master["Complete_Increase"] = master["Appts_Complete_1608"] - master["Appts_Complete_1409"]
master["Complete_Increase_%"] = (master["Complete_Increase"]/master["Appts_Complete_1409"])*100
master["Complete_Pending_Appts_Diff_14"] = master["Appts_Pending_1412"] - master["Appts_Complete_1409"]
master["Complete_Pending_Appts_Diff_16"] = master["Appts_Pending_1610"] - master["Appts_Complete_1608"]

master["Complete_Pending_Appts_Diff_14_%"] = ((master["Appts_Pending_1412"] - master["Appts_Complete_1409"])/master["Appts_Complete_1409"])*100
master["Complete_Pending_Appts_Diff_16_%"] = ((master["Appts_Pending_1610"] - master["Appts_Complete_1608"])/master["Appts_Complete_1608"])*100




master["Complete_Pending_Gap_Increase_%"] = (master["Complete_Pending_Appts_Diff_16"] -
                                             master["Complete_Pending_Appts_Diff_14"]) / master["Complete_Pending_Appts_Diff_14"]

# Leadership
master["Missing_Leadership"] = master["Vacancy"] + master["Interim"]

#Staff
master["Staff_11_To_12_Increase"] = master["MAY-FY12"] - master["MAY-FY11"]
master["Staff_11_To_12_Increase_%"] = (master["Staff_11_To_12_Increase"]/master["MAY-FY12"])*100

master["Staff_12_To_13_Increase"] = master["MAY-FY13"] - master["MAY-FY12"]
master["Staff_12_To_13_Increase_%"] = (master["Staff_12_To_13_Increase"]/master["MAY-FY13"])*100

master["Staff_13_To_14_Increase"] = master["MAY-FY14"] - master["MAY-FY13"]
master["Staff_13_To_14_Increase_%"] = (master["Staff_13_To_14_Increase"]/master["MAY-FY14"])*100

master["Staff_14_To_15_Increase"] = master["MAY-FY15"] - master["MAY-FY14"]
master["Staff_14_To_15_Increase_%"] = (master["Staff_14_To_15_Increase"]/master["MAY-FY15"])*100

master["Staff_15_To_16_Increase"] = master["MAY-FY16"] - master["MAY-FY15"]
master["Staff_15_To_16_Increase_%"] = (master["Staff_15_To_16_Increase"]/master["MAY-FY16"])*100

master["Staff_14_To_16_Increase"] = master["MAY-FY16"] - master["MAY-FY14"]
master["Staff_14_To_16_Increase_%"] = (master["Staff_14_To_16_Increase"]/master["MAY-FY16"])*100

# Hires
master["Choice_Increase_%"] = master["Choice_Hires"]/master["MAY-FY14"]
master["Choice_Staff_Dif"] = master["Staff_14_To_16_Increase"] - master["Choice_Hires"] 
master["Choice_Increase_%_PC_Wait_Complete_1409_Ratio"] = master["PC_Wait_Complete_1409"]/master["Choice_Increase_%"]
master["Choice_Increase_%_PC_Wait_Pending_1412_Ratio"] = master["PC_Wait_Pending_1412"]/master["Choice_Increase_%"]

# Wait Times
master["PC_Wait_Pending_Increase"] = master["PC_Wait_Pending_1610"] - master["PC_Wait_Pending_1412"]
master["PC_Wait_Pending_Increase_%"] = (master["PC_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["PC_Wait_Complete_Increase"] = master["PC_Wait_Complete_1608"] - master["PC_Wait_Complete_1409"]
master["PC_Wait_Complete_Increase_%"] = (master["PC_Wait_Complete_Increase"]/master["PC_Wait_Complete_1409"])*100

master["SC_Wait_Pending_Increase"] = master["SC_Wait_Pending_1610"] - master["SC_Wait_Pending_1412"]
master["SC_Wait_Pending_Increase_%"] = (master["SC_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["SC_Wait_Complete_Increase"] = master["SC_Wait_Complete_1608"] - master["SC_Wait_Complete_1409"]
master["SC_Wait_Complete_Increase_%"] = (master["SC_Wait_Complete_Increase"]/master["SC_Wait_Complete_1409"])*100

master["MH_Wait_Pending_Increase"] = master["MH_Wait_Pending_1610"] - master["MH_Wait_Pending_1412"]
master["MH_Wait_Pending_Increase_%"] = (master["MH_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["MH_Wait_Complete_Increase"] = (master["MH_Wait_Complete_1608"] - master["MH_Wait_Complete_1409"])*100
master["MH_Wait_Complete_Increase_%"] = (master["MH_Wait_Complete_Increase"]/master["MH_Wait_Complete_1409"])*100

# Staff size relative to appointments

master["Staff_To_Appts_Pending_14"] = master["MAY-FY14"]/master["Appts_Pending_1412"]
master["Staff_To_Appts_Pending_16"] = master["MAY-FY16"]/master["Appts_Pending_1610"]
master["Staff_To_Appts_Complete_14"] = master["MAY-FY14"]/master["Appts_Complete_1409"]
master["Staff_To_Appts_Complete_16"] = master["MAY-FY16"]/master["Appts_Complete_1608"]


# Appointments Over 30

master["Percentage_Point_Wait_Over_30_Increase_Pending"] = master['%_Appts_Over_30_Pending_1610'] - master['%_Appts_Over_30_Pending_1412']
master["Percentage_Point_Wait_Over_30_Increase_Complete"] = master['%_Appts_Over_30_Complete_1608'] - master['%_Appts_Over_30_Complete_1409']

# Get State

master["State"] = master["Location"].apply(get_state)



# Export to CSV

In [1508]:
master.to_csv("Data/Master/Master.csv", index=False)

In [1509]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'ID', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location', u'for_dups', u'Choice_Hires', u'Interim', u'Vacancy',
       u'%_Appts_Over_30_Complete_1409', u'Appts_Complete_1409',
       u'MH_Wait_Complete_1409', u'PC_Wait_Complete_1409',
       u'SC_Wait_Complete_1409', u'Physician_Turnover_FY11',
       u'Physician_Turnover_FY12', u'Physician_Turnover_FY13',
       u'Physician_Turnover_FY14', u'Physician_Turnover_FY15',
       u'NbrPsychiatrists', u'Pending_In

In [1510]:
for column in master.columns:
    print column

Appts_Pending_1412
%_Appts_Over_30_Pending_1412
PC_Wait_Pending_1412
SC_Wait_Pending_1412
MH_Wait_Pending_1412
ID
ORGANIZATION
MAY-FY11
MAY-FY12
MAY-FY13
MAY-FY14
MAY-FY15
MAY-FY16
Appts_Pending_1610
%_Appts_Over_30_Pending_1610
PC_Wait_Pending_1610
SC_Wait_Pending_1610
MH_Wait_Pending_1610
Appts_Complete_1608
%_Appts_Over_30_Complete_1608
PC_Wait_Complete_1608
SC_Wait_Complete_1608
MH_Wait_Complete_1608
Station
Location
for_dups
Choice_Hires
Interim
Vacancy
%_Appts_Over_30_Complete_1409
Appts_Complete_1409
MH_Wait_Complete_1409
PC_Wait_Complete_1409
SC_Wait_Complete_1409
Physician_Turnover_FY11
Physician_Turnover_FY12
Physician_Turnover_FY13
Physician_Turnover_FY14
Physician_Turnover_FY15
NbrPsychiatrists
Pending_Increase
Pending_Increase_%
Complete_Increase
Complete_Increase_%
Complete_Pending_Appts_Diff_14
Complete_Pending_Appts_Diff_16
Complete_Pending_Appts_Diff_14_%
Complete_Pending_Appts_Diff_16_%
Complete_Pending_Gap_Increase_%
Missing_Leadership
Staff_11_To_12_Increase
Staff_1

# National

In [1511]:
#Taken from VA site
national_complete = pd.read_csv("Data/Wait_Time/Completed_Cleaned/National_Complete.csv")
national_pending = pd.read_csv("Data/Wait_Time/Pending_Clean/National_Pending.csv")


#used to strip time from Pending_Time
def get_date(test):
    test = test.strip('National: ')
    return parser.parse(test)

national_complete["date"] = national_complete["Complete_Time"].apply(get_date)
national_pending["date"] = national_pending["Pending_Time"].apply(get_date)
national_complete.sort("date")
national_pending.sort("date")

national_pending = national_pending.drop(20) #for some reason this was read in as 1989. Just dropping it

#national_pending.to_csv("National/National_Pending.csv",index=False)
#national_complete.to_csv("National/National_Complete.csv",index=False)

#NC = national_complete[["Appts_Complete_National","date"]]
#NP = national_pending[["Appts_Pending_National","date"]]

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
